# Processing

In [ ]:
import numpy as np
import pandas as pd
import os

#### Directories

In [ ]:
ROOT_DIR = os.path.abspath('..')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

CUSTOMER_DIR = os.path.join(DATA_DIR, '2_detail_table_customers.xls')
VEHICLES_DIR = os.path.join(DATA_DIR, '3_detail_table_vehicles.xls')
DEPOTS_DIR = os.path.join(DATA_DIR, '4_detail_table_depots.xls')
CONSTRAINTS_DIR = os.path.join(DATA_DIR, '5_detail_table_constraints_sdvrp.xls')
DEPOTS_DISTANCES_DIR = os.path.join(DATA_DIR, '6_detail_table_cust_depots_distances.xls')
CUSTOMER_DISTANCES_DIR = os.path.join(DATA_DIR, '7_detail_table_cust_cust_distances.xls')

In [3]:
customers = pd.read_excel(CUSTOMER_DIR)
vehicles = pd.read_excel(VEHICLES_DIR)
depots = pd.read_excel(DEPOTS_DIR)
constraints = pd.read_excel(CONSTRAINTS_DIR)
depots_dist = pd.read_excel(DEPOTS_DISTANCES_DIR)
customers_dist = pd.read_excel(CUSTOMER_DISTANCES_DIR)

In [ ]:
def load_vehicle(vehicles, MODE):
    if MODE == "mean":
        volume = vehicles["VEHICLE_TOTAL_VOLUME_M3"].mean()
        weight = vehicles["VEHICLE_TOTAL_WEIGHT_KG"].mean()
        cost_km = vehicles["VEHICLE_VARIABLE_COST_KM"].mean()
    if MODE == "min":
        volume = vehicles["VEHICLE_TOTAL_VOLUME_M3"].min()
        weight = vehicles["VEHICLE_TOTAL_WEIGHT_KG"].min()
        cost_km = vehicles["VEHICLE_VARIABLE_COST_KM"].min()  
    if MODE == "max":
        volume = vehicles["VEHICLE_TOTAL_VOLUME_M3"].max()
        weight = vehicles["VEHICLE_TOTAL_WEIGHT_KG"].max()
        cost_km = vehicles["VEHICLE_VARIABLE_COST_KM"].max()
    vehicle = Vehicle(volume, weight, cost_km)
    return vehicle

### REVISAR EL ORDEN DE LOS CUSTOMERS

In [ ]:
def data_from_route(path, route_id):
    df = pd.read_excel(path)
    return df[df['ROUTE_ID'] == route_id].drop(['ROUTE_ID'], axis=1)

def matrix_generator(depot_data, customer_data):
    n = len(depot_data)//2
    depot_data.sort_values(['CUSTOMER_CODE']).reset_index(drop=True, inplace=True)
    customer_data.sort_values(['CUSTOMER_CODE_FROM', 'CUSTOMER_CODE_TO']).reset_index(drop=True, inplace=True)
    time_matrix = np.zeros((n+1,n+1))
    distance_matrix = np.zeros((n+1,n+1))
    groups_depot = dict(tuple(depot_data.groupby(['DIRECTION'])))
    time_matrix[0, 1:] = groups_depot['DEPOT->CUSTOMER']['TIME_DISTANCE_MIN'].to_numpy()
    time_matrix[1:, 0] = groups_depot['CUSTOMER->DEPOT']['TIME_DISTANCE_MIN'].to_numpy()
    distance_matrix[0, 1:] = groups_depot['DEPOT->CUSTOMER']['DISTANCE_KM'].to_numpy()
    distance_matrix[1:, 0] = groups_depot['CUSTOMER->DEPOT']['DISTANCE_KM'].to_numpy()
    groups_customer = dict(tuple(customer_data.groupby(['CUSTOMER_CODE_FROM'])))
    keys = np.array(list(groups_customer.keys()))
    for i in range(1, n+1):
        time_matrix[i, 1:] = groups_customer[keys[i-1]]['TIME_DISTANCE_MIN'].to_numpy()
        distance_matrix[i, 1:] = groups_customer[keys[i-1]]['DISTANCE_KM'].to_numpy()
    return time_matrix, distance_matrix, keys